# 模仿《安娜卡列尼娜》的文本生成代码，构建一个平凡的世界的生成模型

In [ ]:
import time
from collections import namedtuple
import numpy as np
import pandas as pd
import jieba
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## 1 数据加载与预处理

In [ ]:
with open ('./平凡的世界.txt') as f:
    text=f.read()
# 对文本进行结巴分词处理
text=''.join(list(jieba.lcut(text)))
vocab_sentence=[]
for sentence in text:
    vocab_sentence.append(text)

In [ ]:
#   获取分词后的文本
vocab=[j for i in vocab_sentence for j in i]
# 获取文章中出现的所有分词
vocab_set=set(vocab)
vocab_to_int={c:i for i,c in enumerate(vocab_set)}
int_to_vocab=dict(enumerate(vocab_set))
# 将分词后的文本转化为整数
encoded=np.array([vocab_to_int[c] for c in vocab],dtype=np.int32)

## 2 数据集划分

#### 我们定义一个batch中的序列个数为N（batch_size），定义单个序列长度为M（也就是我们的steps）。那么实际上我们每个batch是一个N x M的数组。在这里我们重新定义batch_size为一个N x M的数组，而不是batch中序列的个数。在上图中，当我们设置N=2， M=3时，我们可以得到每个batch的大小为2 x 3 = 6个字符，整个序列可以被分割成12 / 6 = 2个batch。

In [ ]:
def get_batches(arr,n_seqs,n_steps):
    '''
    对已有的数组进行mini-batch划分
    arr:带划分的数组
    n_seqs:一个batch中的序列个数
    n_steps:单个序列包含的字符数
    '''
    batch_size=n_seqs * n_steps
    n_batches=int(len(arr) / batch_size)
#     保留完整的batch
    arr=arr[:batch_size * n_batches]
    arr=arr.shape(n_seqs,-1)
    
    for n in range(0,arr.shape[1],n_steps):
#         inputs
    x=arr[:,n:n+n_steps]
#     target
    y=[:,:-1],y[:,-1]=x[:,1],x[:,0]
    
    yield x,y

In [ ]:
batches=get_batches(encoded,10,50)
x,y=next(batches)

In [ ]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])
print(x.shape)
print(y.shape)

## 3 模型构建
####  模型构建部分主要包括了输入层、LSTM层、输出层、loss、optimize等部分的构建

### 3.1 输入层

In [ ]:
def build_inputs(num_seqs,num_steps):
    '''
    构建输入层
    
    num_seqs:每个batch中的序列个数
    num_steps:每个序列包含的字符数
    '''
    inputs=tf.placeholder(tf.int32,shape=(num_seqs,num_steps),name='inputs')
    targets=tf.placeholder(tf.int32,shape=(num_seqs,num_steps),name='targets')  
    
#     加入keep_prob：dropout中保留的概率
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    
    return inputs,targets,keep_prob    

### 3.2 LSTM层

In [ ]:
def build_lstm(lstm_size,num_layers,batch_size,keep_prob):
    '''
    构建lstm层
    
    keep_prob:dropout保留的概率
    num_layers:lstm的隐藏层的数目
    batch_size
    '''
    def get_a_cell(lstm_size,keep_prob):
        lstm=tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
        drop=tf.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
        return drop
    
#     构建一个基本的lstm单元
    lstm=tf.nn.rnn_cell.BasicLSTMCell(lstm_size) 
    
#     添加dropout
    drop=tf.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    
#     堆叠:构建多隐层神经网络
# 使用RNN堆叠函数将前面构造的lstm_cell多层堆叠得到cell，堆叠次数为lstm中隐层数目-num_layers
    cell=tf.nn.rnn_cell.MultiRNNCell([get_a_cell(lstm_size,keep_prob)]) for _ in range(num_layers)
#     设置LSTM单元的初始化状态为0
    initial_state=cell.zero_state(batch_size,tf.float32)
    
    return cell,initial_state

### 3.3 输出层

In [ ]:
def build_output(lstm_output,in_size,out_size):
    '''
    构造输出层
    
    lstm_output:lstm层的输出结果
    in_size:lstm输出层重塑后的size
    out_size:softmax层的size
    '''
    
    seq_output=tf.concat(lstm_output,1)
    x=tf.reshape(seq_output,[-1,in_size])
    
#     将lstm层和softmax层全连接起来
    with tf.variable_scope('softmax'):
        softmax_w=tf.Variable(tf.trun)